In [1]:
import numpy as np

def _get_permutation_map_closedshell_convention(element_numbers):
    """
    Permutation map for converting the matrices obtained from `tblite`
    GFN-xTB (closed shell) to the convention for orbnet-equi.

    Input (dxtb) convention:
     - H: [1s, 2s]
     - C,N,O,F: [2s, 2pz, 2px, 2py]
    Output (tblite) convention:
     - H: [1s, 2s] (no change)
     - C,N,O,F: [2s, 2px, 2py, 2pz]
    """
    n_so = 0
    for el in element_numbers:
        if el == 1:
            n_so += 2
        else:
            n_so += 4

    perm_map = np.zeros(n_so, dtype=int)
    idx = 0
    for el in element_numbers:
        if el == 1:
            pmap = [idx, idx + 1]
            perm_map[idx : idx + 2] = pmap
            idx += 2
        else:
            # [s, pz, px, py] -> [s, px, py, pz]
            pmap = [idx, idx + 2, idx + 3, idx + 1]
            perm_map[idx : idx + 4] = pmap
            idx += 4
    return perm_map

def _get_permutation_map_closedshell_convention_dxtb_to_OrbNet(element_numbers):
    """
    Permutation map for converting the matrices obtained from `tblite`
    GFN-xTB (closed shell) to the convention for orbnet-equi.

    Input (dxtb) convention:
     - H: [1s, 2s]
     - C,N,O,F: [2s, 2pz, 2px, 2py]
    Output (orbnet-equi) convention:
     - H: [1s, 2s] (no change)
     - C,N,O,F: [2s, 2pz, 2py, 2px]
    """
    n_so = 0
    for el in element_numbers:
        if el == 1:
            n_so += 2
        else:
            n_so += 4

    perm_map = np.zeros(n_so, dtype=int)
    idx = 0
    for el in element_numbers:
        if el == 1:
            pmap = [idx, idx + 1]
            perm_map[idx : idx + 2] = pmap
            idx += 2
        else:
            # [s, pz, px, py] -> [s, pz, py, px]
            pmap = [idx, idx + 1, idx + 3, idx + 2]
            perm_map[idx : idx + 4] = pmap
            idx += 4
    return perm_map

def apply_perm_map(mat, perm_map):
    ndim = mat.ndim
    if ndim == 1:
        matp = mat[perm_map]
    elif ndim == 2:
        matp = mat[perm_map, :]
        matp = matp[:, perm_map]
    else:
        raise ValueError("Only 1D and 2D arrays are supported.")
    return matp

In [2]:
import torch
import dxtb
from dxtb.typing import DD
from dxtb.config import ConfigCache
from dxtb.integrals.wrappers import overlap, dipint, hcore

dd: DD = {"dtype": torch.double, "device": torch.device("cpu")}

# numbers = torch.tensor([6, 6, 7, 7, 1, 1, 1, 1, 1, 1, 8, 8,], device=dd["device"])
# positions = torch.tensor([
#                 [-3.81469488143921, +0.09993441402912, 0.00000000000000],
#                 [+3.81469488143921, -0.09993441402912, 0.00000000000000],
#                 [-2.66030049324036, -2.15898251533508, 0.00000000000000],
#                 [+2.66030049324036, +2.15898251533508, 0.00000000000000],
#                 [-0.73178529739380, -2.28237795829773, 0.00000000000000],
#                 [-5.89039325714111, -0.02589114569128, 0.00000000000000],
#                 [-3.71254944801331, -3.73605775833130, 0.00000000000000],
#                 [+3.71254944801331, +3.73605775833130, 0.00000000000000],
#                 [+0.73178529739380, +2.28237795829773, 0.00000000000000],
#                 [+5.89039325714111, +0.02589114569128, 0.00000000000000],
#                 [-2.74426102638245, +2.16115570068359, 0.00000000000000],
#                 [+2.74426102638245, -2.16115570068359, 0.00000000000000],
#                 ], **dd) # ** to use dd as kwargs

numbers = torch.tensor([3, 1], device=dd["device"])
positions = torch.tensor([[0.0, 0.0, 0.0], [0.0, 0.0, 1.5]], **dd) # ** to use dd as kwargs 

# numbers = torch.tensor([1, 3], device=dd["device"])
# positions = torch.tensor([[0.0, 0.0, 1.5], [0.0, 0.0, 0.0]], **dd) # ** to use dd as kwargs 

# numbers = torch.tensor([3], device=dd["device"])
# positions = torch.tensor([[0.0, 0.0, 0.0]], **dd) # ** to use dd as kwargs

# numbers = torch.tensor([6, 1, 1, 8, 9], device=dd["device"])
# positions = torch.tensor([
#     [0.0, 0.0, 0.0],      # C (Carbon)
#     [1.0, 0.0, 0.0],      # H (Hydrogen)
#     [-1.0, 0.0, 0.0],     # H (Hydrogen)
#     [0.0, 1.0, 0.0],      # O (Oxygen)
#     [0.0, -1.0, 0.0]      # F (Fluorine)
# ], **dd)

cache_config = ConfigCache(density=True, coefficients=True, mo_energies=True)
calc = dxtb.Calculator(numbers, dxtb.GFN1_XTB, **dd)
calc.opts.cache = cache_config

# compute the energy
pos = positions.clone().requires_grad_(True)
energy = calc.get_energy(pos, verbosity = 0)
density_spinNone = calc.get_density(pos, spin=None)
s = overlap(numbers, pos, dxtb.GFN1_XTB)

ModuleNotFoundError: No module named 'dxtb'

In [3]:
from tblite.interface import Calculator
import numpy as np

calc_tblite = Calculator(
    method = "GFN1-xTB",
    numbers = numbers.numpy(),
    positions = positions.numpy(),
    uhf=None
)

# calc_tblite.add("spin-polarization", 1.0)
calc_tblite.set("save-integrals", 1)
res = calc_tblite.singlepoint()

P = res["density-matrix"]
S = res["overlap-matrix"]

print(f"P.shape = {P.shape}")
print(f"S.shape = {S.shape}")
print(f"e_tbl - e_dxtb = {res['energy'] - energy.detach().numpy()}, energy_tblite = {res['energy']}, energy_dxtb = {energy}")

perm_map = _get_permutation_map_closedshell_convention(numbers.numpy())

p_perm = apply_perm_map(density_spinNone.detach().numpy(), perm_map)
s_perm = apply_perm_map(s.detach().numpy(), perm_map)

tol = 1e-3
# print(f"np.allclose(P, p_perm) (tol {tol:.0e}) = {np.allclose(P, p_perm, atol=tol)}")
# print(f"np.allclose(P[0] + P[1], p_perm) (tol {tol:.0e}) = {np.allclose(P[0] + P[1], p_perm, atol=tol)}")
print(f"np.allclose(S, s_perm) (tol {tol:.0e}) = {np.allclose(S, s_perm, atol=tol)}")

------------------------------------------------------------
  cycle        total energy    energy error   density error
------------------------------------------------------------
      1    -0.6739139343013  -8.9186820E-01   3.4717875E-01
      2    -0.6775793340317  -3.6653997E-03   1.6832813E-01
      3    -0.6785744597801  -9.9512575E-04   1.9516652E-02
      4    -0.6785806864440  -6.2266640E-06   1.7840870E-02
      5    -0.6786026214695  -2.1935026E-05   7.0178883E-04
      6    -0.6786026566184  -3.5148836E-08   3.5195037E-06
------------------------------------------------------------

 total:                                   0.002 sec
P.shape = (6, 6)
S.shape = (6, 6)
e_tbl - e_dxtb = -3.979261398168177e-08, energy_tblite = -0.6786026566183807, energy_dxtb = -0.6786026168257667
np.allclose(S, s_perm) (tol 1e-03) = True


In [9]:
from tblite.library import get_gradient

res.get("gradient")

array([[-6.04570483e-17,  0.00000000e+00,  6.33466895e-01],
       [ 6.04570483e-17,  0.00000000e+00, -6.33466895e-01]])

# Permutation map

In [34]:
perm_map = _get_permutation_map_closedshell_convention(numbers.numpy())

s_perm = apply_perm_map(s.detach().numpy(), perm_map)
p_perm = apply_perm_map(density_spinNone.detach().numpy(), perm_map)

np.set_printoptions(precision=3, suppress=True, linewidth=1000, threshold=np.inf)

# print(f"s_perm: \n {s_perm}")
# print(f"S: \n {S}")
# print()
# print(f"p_perm: \n {p_perm}")
# print(f"P: \n {P}")

# F matrices

In [35]:
e = calc.get_mo_energies(pos)
c = calc.get_coefficients(pos)

E = res["orbital-energies"]
C = res["orbital-coefficients"]

Total Energy: -0.67860261682577 Hartree.
Total Energy: -0.67860261682577 Hartree.


In [36]:
import numpy.linalg as LA

F = S.dot(C).dot(np.diag(E)).dot(LA.inv(C))

f = s.detach().numpy().dot(c.detach().numpy()).dot(np.diag(e.detach().numpy())).dot(LA.inv(c.detach().numpy()))
f_perm = apply_perm_map(f, perm_map)

print(f"np.allclose(F, f_perm) (tol {tol:.0e}) = {np.allclose(F, f_perm, atol=tol)}")

np.allclose(F, f_perm) (tol 1e-03) = True


In [37]:
e = calc.get_mo_energies(pos)
c = calc.get_coefficients(pos)

E = res["orbital-energies"]
C = res["orbital-coefficients"]

print(f"e.shape {e.shape}, E.shape {E.shape}")
print(f"c.shape {c.shape}, C.shape {C.shape}")

e_perm = apply_perm_map(e.detach().numpy(), perm_map)
c_perm = apply_perm_map(c.detach().numpy(), perm_map)

tol = 1e-1
print(f"np.allclose(e, E) (tol {tol:.0e}) = {np.allclose(e.detach().numpy(), E, atol=tol)}")
# print(f"np.allclose(c, C) (tol {tol:.0e}) = {np.allclose(c_perm, C, atol=tol)}")

print(f"e: {e.detach().numpy()}")
print(f"E: {E}")

Total Energy: -0.67860261682577 Hartree.
Total Energy: -0.67860261682577 Hartree.


e.shape torch.Size([6]), E.shape (6,)
c.shape torch.Size([6, 6]), C.shape (6, 6)
np.allclose(e, E) (tol 1e-01) = True
e: [-0.413 -0.205 -0.164 -0.164 -0.024  0.374]
E: [-0.413 -0.205 -0.164 -0.164 -0.024  0.374]


In [38]:
print(f"c: \n{c.detach().numpy()}")
print(f"C: \n{C}")
print(f"c_perm: \n{c_perm}")

c: 
[[ 0.376 -0.656  0.    -0.    -0.003 -1.061]
 [ 0.     0.     0.348 -0.937 -0.     0.   ]
 [-0.    -0.    -0.937 -0.348  0.     0.   ]
 [ 0.182  0.717 -0.     0.    -0.362 -0.755]
 [ 0.674  0.204 -0.    -0.     0.237  1.145]
 [ 0.01   0.131  0.    -0.     0.937 -0.435]]
C: 
[[-0.376  0.656 -0.     0.    -0.003 -1.061]
 [ 0.     0.     0.     1.     0.     0.   ]
 [-0.182 -0.717  0.     0.    -0.362 -0.755]
 [ 0.    -0.    -1.     0.     0.     0.   ]
 [-0.674 -0.204  0.     0.     0.237  1.145]
 [-0.01  -0.131  0.     0.     0.937 -0.435]]
c_perm: 
[[ 0.376  0.    -0.    -0.656 -0.003 -1.061]
 [-0.    -0.937 -0.348 -0.     0.     0.   ]
 [ 0.182 -0.     0.     0.717 -0.362 -0.755]
 [ 0.     0.348 -0.937  0.    -0.     0.   ]
 [ 0.674 -0.    -0.     0.204  0.237  1.145]
 [ 0.01   0.    -0.     0.131  0.937 -0.435]]
